# WIP: Video generation

In [1]:
# General

import numpy
import pandas

# Audio

from sunflower.sunflower.song_loader import Song, load_from_disk
from sunflower.sunflower.song_analyzer import SongAnalyzer
from sunflower.sunflower.utils import export_wav
from sunflower.sunflower.song_visualizer import visualize_waveform
import plotly.graph_objects as go
import numpy as np
import librosa
import soundfile as sf

# Video

from moviepy.editor import *
from moviepy.audio.AudioClip import AudioArrayClip
import moviepy
import pygame

# Autoreloading for easier development

%load_ext autoreload
%autoreload 2


ModuleNotFoundError: No module named 'matplotlib'

## Audio

In [ ]:
# Loading example file

raw_audio, extension = load_from_disk("assets/laylow_casting_slowed.mp3")

song = Song(raw_audio, extension)

song.print_attributes()

In [ ]:
# Analyzing song

song_analyzer = SongAnalyzer(song)
song_analyzer.detect_tempo()

print(song_analyzer.tempo)

## Video

In [ ]:
def squared_size(clip): 
    
    # Resize
    clip=clip.crop(x_center=int(clip.w/2), 
                   y_center=int(clip.h/2),
                   width=min(clip.w,clip.h), 
                   height=min(clip.w,clip.h))
    
    return(clip)

def invert_green_blue(image):
    return image[:,:,[0,1,1]]



In [ ]:
audioclip = AudioArrayClip(song.waveform , 
                           fps=song.sr)

In [ ]:
# Settings

WIDTH = 600
HEIGHT = 600

screensize = (WIDTH,HEIGHT)
prop_center = 0.8

BPM = 112
duration = 60/BPM

artist_name = "Laylow"
track_name = "Casting"

# Fonts

textwidth = 0.2*WIDTH
textheight = 0.2*HEIGHT

In [ ]:
txtClip_artist = TextClip(artist_name,color='white', font="Helvetica",
                   kerning = 5, size=(textwidth,None))

txtClip_track = TextClip(track_name,color='LightPink', font="Helvetica-bold",
                   kerning = 5, size=(textwidth,None))

# Load all the videos

# Zankyou

clip_og = VideoFileClip("assets/zankyou.mp4")
clip_og=squared_size(clip_og)
clip_og=clip_og.resize([int(size*prop_center) for size in screensize])

# VHS

clip_vhs = VideoFileClip("assets/vhs.mp4")
clip_vhs=squared_size(clip_vhs)
clip_vhs=clip_vhs.resize(screensize)

masked_clip = clip_vhs.subclip(500,550).fx(moviepy.editor.vfx.mask_color, color=[0, 0, 0], thr=100, s=5)

# Background

clip_background = VideoFileClip("assets/vhs.mp4")
clip_background =squared_size(clip_background)
clip_background=clip_background.resize(screensize)

# Son

clip = clip_vhs.subclip(120,120+4*duration)

cuts = [51,53]


clip=concatenate_videoclips([clip,clip_og.subclip(cuts[0],cuts[0]+duration).loop(duration=duration*28)])
clip=concatenate_videoclips([clip,clip_og.subclip(cuts[1],cuts[1]+duration).loop(duration=duration*28)])
    

# Looping the clip
clip = clip.loop(n=2)

# Say that you want it to appear 10s at the center of the screen
txtClip_artist = txtClip_artist.set_pos((WIDTH/2 - txtClip_artist.w/2,0.1*HEIGHT)).set_start(duration*2).set_duration(duration)

# Say that you want it to appear 10s at the center of the screen
txtClip_track = txtClip_track.set_pos((WIDTH/2 - txtClip_track.w/2,0.25*HEIGHT)).set_start(duration*2).set_duration(duration)


# Overlay the text clip on the first video clip
video = CompositeVideoClip([clip, txtClip_artist, txtClip_track,masked_clip])

video=video.set_audio(audioclip)



# Cut de la video 

video = video.subclip(0,50)


In [ ]:
video.preview(fps=10)

In [ ]:
pygame.quit()

In [ ]:
#video.ipython_display(fps=20, autoplay=1,maxduration=30)

## Saving video

In [ ]:
video.write_videofile("test2.mp4", temp_audiofile="temp-audio.m4a", remove_temp=True, codec="libx264", audio_codec="aac")